In [1]:
from google.colab import drive
drive.mount("/gdrive")
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
import networkx
from operator import itemgetter
import matplotlib.pyplot
import pandas as pd
import numpy as np

In [0]:
# Read the data from amazon-books.csv into amazonBooks dataframe;
amazonBooks = pd.read_csv('/gdrive/My Drive/CIS_508/Colab Notebooks/CIS_509/Asmt3/amazon-books.csv', index_col=0)

# Read the data from amazon-books-copurchase.adjlist;
# assign it to copurchaseGraph weighted Graph;
# node = ASIN, edge= copurchase, edge weight = category similarity

fhr=open("/gdrive/My Drive/CIS_508/Colab Notebooks/CIS_509/Asmt3/amazon-books-copurchase.edgelist", 'rb')
copurchaseGraph=networkx.read_weighted_edgelist(fhr)
fhr.close()

In [4]:
amazonBooks.sample(5)

,Id,Title,Categories,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
1565542983,282800,Do I Have to Draw You a Picture,general novels entertainment comics graphic su...,1487262,0,0.0,0,0.000000
0801866200,197605,"Mandate Days : British Lives in Palestine, 191...",general history spirituality israel religion j...,901461,0,0.0,2,0.000000
080696877X,536490,Handmade Paper Collage,art general papercrafts hobbies mixed home cra...,683322,1,5.0,3,0.333333
0939650444,239794,Child of Hitler: Germany in the Days When God ...,general history memoirs subjects books histori...,154194,8,5.0,2,1.000000
061313169X,179725,Who Let Girls in the Boys Locker Room?,NaN,1909967,0,0.0,0,0.000000


In [5]:
amazonBooks.shape

(393560, 8)

In [6]:
amazonBooks.describe()

,Id,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
count,393560.000000,3.935600e+05,393560.000000,393560.000000,393560.000000,393560.000000
mean,272718.261399,6.309075e+05,11.961533,3.051143,3.766216,0.276585
std,156441.922102,6.044106e+05,72.982795,2.064086,5.445714,0.334681
min,2.000000,-1.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,137126.750000,2.034860e+05,0.000000,0.000000,0.000000,0.000000
50%,274282.500000,4.672215e+05,2.000000,4.000000,3.000000,0.133333
75%,408082.250000,8.545142e+05,6.000000,5.000000,5.000000,0.500000
max,548549.000000,3.798351e+06,5545.000000,5.000000,324.000000,1.000000


In [7]:
# Now let's assume a person is considering buying the following book;
# what else can we recommend to them based on copurchase behavior 
# we've seen from other users?

print ("Looking for Recommendations for Customer Purchasing this Book:")
print ("--------------------------------------------------------------")
purchasedAsin = '0805047905'

# Let's first get some metadata associated with this book

print ("ASIN = ", purchasedAsin) 
print ("Title = ", amazonBooks.loc[purchasedAsin,'Title'])
print ("SalesRank = ", amazonBooks.loc[purchasedAsin,'SalesRank'])
print ("TotalReviews = ", amazonBooks.loc[purchasedAsin,'TotalReviews'])
print ("AvgRating = ", amazonBooks.loc[purchasedAsin,'AvgRating'])
print ("DegreeCentrality = ", amazonBooks.loc[purchasedAsin,'DegreeCentrality'])
print ("ClusteringCoeff = ", amazonBooks.loc[purchasedAsin,'ClusteringCoeff'])
    

Looking for Recommendations for Customer Purchasing this Book:
--------------------------------------------------------------
ASIN =  0805047905
Title =  Brown Bear, Brown Bear, What Do You See?
SalesRank =  171
TotalReviews =  172
AvgRating =  5.0
DegreeCentrality =  216
ClusteringCoeff =  0.01976744186046512


In [0]:
# Now let's look at the ego network associated with purchasedAsin in the
# copurchaseGraph - which is esentially comprised of all the books 
# that have been copurchased with this book in the past
# (1) YOUR CODE HERE: 
#     Get the depth-1 ego network of purchasedAsin from copurchaseGraph,
#     and assign the resulting graph to purchasedAsinEgoGraph.

purchasedAsinEgoGraph = networkx.Graph()
purchasedAsinEgoGraph = networkx.ego_graph(copurchaseGraph, purchasedAsin, radius = 1)
#purchasedAsinEgoGraph

In [0]:
# Next, recall that the edge weights in the copurchaseGraph is a measure of
# the similarity between the books connected by the edge. So we can use the 
# island method to only retain those books that are highly simialr to the 
# purchasedAsin
# (2) YOUR CODE HERE: 
#     Use the island method on purchasedAsinEgoGraph to only retain edges with 
#     threshold >= 0.5, and assign resulting graph to purchasedAsinEgoTrimGraph

threshold = 0.5
purchasedAsinEgoTrimGraph = networkx.Graph()
for f,t,e in purchasedAsinEgoGraph.edges(data= True):
  if e['weight'] >= threshold:
    purchasedAsinEgoTrimGraph.add_edge(f,t,weight = e['weight'])
    #print(f,t,e)

In [10]:
# Next, recall that given the purchasedAsinEgoTrimGraph you constructed above, 
# you can get at the list of nodes connected to the purchasedAsin by a single 
# hop (called the neighbors of the purchasedAsin) 
# (3) YOUR CODE HERE: 
#     Find the list of neighbors of the purchasedAsin in the 
#     purchasedAsinEgoTrimGraph, and assign it to purchasedAsinNeighbors

purchasedAsinNeighbors = []
ego1 = networkx.ego_graph(purchasedAsinEgoTrimGraph, purchasedAsin, radius = 1)
purchasedAsinNeighbors = [i for i in ego1.neighbors(purchasedAsin)]
len(purchasedAsinNeighbors)

41

In [0]:
# Next, let's pick the Top Five book recommendations from among the 
# purchasedAsinNeighbors based on one or more of the following data of the 
# neighboring nodes: SalesRank, AvgRating, TotalReviews, DegreeCentrality, 
# and ClusteringCoeff
# (4) YOUR CODE HERE: 
#     Note that, given an asin, you can get at the metadata associated with  
#     it using amazonBooks (similar to lines 29-36 above).
#     Now, come up with a composite measure to make Top Five book 
#     recommendations based on one or more of the following metrics associated 
#     with nodes in purchasedAsinNeighbors: SalesRank, AvgRating, 
#     TotalReviews, DegreeCentrality, and ClusteringCoeff. Feel free to compute
#     and include other measures if you like.
#     YOU MUST come up with a composite measure.
#     DO NOT simply make recommendations based on sorting!!!
#     Also, remember to transform the data appropriately using 
#     sklearn preprocessing so the composite measure isn't overwhelmed 
#     by measures which are on a higher scale.

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
amazonBooks = amazonBooks.dropna() #categories has nan columns we drop them as we do not want to make recommendations that have NaN in categories column

mms_salesRank = MinMaxScaler(feature_range=(1,2))
amazonBooks['mms_SalesRank'] = mms_salesRank.fit_transform(amazonBooks[['SalesRank']])

mms_totalReviews = MinMaxScaler(feature_range=(1,2))
amazonBooks['mms_TotalReviews'] = mms_totalReviews.fit_transform(amazonBooks[['TotalReviews']])

amazonBooks['new_AvgRating'] = amazonBooks['AvgRating']+1

mms_degreeCentrality = MinMaxScaler(feature_range=(1,2))
amazonBooks['mms_DegreeCentrality'] = mms_degreeCentrality.fit_transform(amazonBooks[['DegreeCentrality']])

amazonBooks['new_ClusteringCoeff'] = amazonBooks['ClusteringCoeff']+1

amazonBooks.describe()

,Id,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff,mms_SalesRank,mms_TotalReviews,new_AvgRating,mms_DegreeCentrality,new_ClusteringCoeff
count,375138.000000,3.751380e+05,375138.000000,375138.000000,375138.000000,375138.000000,375138.000000,375138.000000,375138.000000,375138.000000,375138.000000
mean,273376.904584,5.977189e+05,12.548931,3.200977,3.893783,0.283952,1.157363,1.002263,4.200977,1.012018,1.283952
std,156766.793607,5.640271e+05,74.703997,1.997512,5.521098,0.335464,0.148493,0.013472,1.997512,0.017040,0.335464
min,2.000000,-1.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,137484.250000,1.959880e+05,0.000000,0.000000,0.000000,0.000000,1.051598,1.000000,1.000000,1.000000,1.000000
50%,274996.000000,4.504300e+05,2.000000,4.000000,3.000000,0.166667,1.118586,1.000361,5.000000,1.009259,1.166667
75%,409086.750000,8.133192e+05,7.000000,5.000000,5.000000,0.500000,1.214125,1.001262,6.000000,1.015432,1.500000
max,548549.000000,3.798351e+06,5545.000000,5.000000,324.000000,1.000000,2.000000,2.000000,6.000000,2.000000,2.000000


In [0]:
AsinNeighborsData = pd.DataFrame()
AsinNeighborsData = amazonBooks.loc[purchasedAsinNeighbors,:]

# **Composite Score Calculation:** 

The idea is to recommend the books that have higher composite score. 

Composite score is..


1. Inversely proportional to SalesRank - Higher the Rank, lesser the score. Books with smaller sales rank have been sold the most, and we also want to recommend books that are best selling which means that lot of people like and buy that book. 
SalesRank is scaled to (1,2) 

2. Directly proportional to Total Reviews - Higher the number of Reviews the better it is to recommend a book, so a higher composite score as well. TotalReviews are scaled to (1,2)

3. Directly proportional to AvgRating - Higher the rating the better it is to recommend that book, so a higher composite score. 
AvgRating is scaled to (1,6)

4. Directly proportional to DegreeCentrality - Higher the degree centrality more similar books it is connected to. So higher the composite score. DegreeCentrality is scaled to (1,2)

5. Directly proportional to the Clustering Coefficient - Higher the clustiering coefficient more strong ties the book's ego network has. So it is highly likely that the book is from the same genre and category as the purchasedAsin. So higher the composite score as well. 
ClusteringCoefficient is scaled to (1,2) 

**Composite score = (DegCentrality x AvgRating x ClusteringCoeff x TotalReviews)/(SalesRank)**

Composite score ranges from 1 to 24. 

Minimum value of all factors in the composite score is 1, it is designed in such a way that even if one factor goes to its minimum value, it doesn't affect the other factors and doesn't pull down the composite score to zero.

**RecommendationScore = Composite Score scaled to (0,1)**. **We sort the AsinNeighbours in descending order of Recommendation score and recommend top N books.**

In [0]:
def getCompositeScore(df):
  salesRank = df['mms_SalesRank']
  degCentrality = df['mms_DegreeCentrality']
  avgRating = df['new_AvgRating']
  clusteringCoeff = df['new_ClusteringCoeff']
  totalReviews = df['mms_TotalReviews']
  compositeScore = (degCentrality*avgRating*clusteringCoeff*totalReviews)/(salesRank)
  df['CompositeScore'] = compositeScore
  mms = MinMaxScaler()
  df['RecommendationScore'] = mms.fit_transform(df[['CompositeScore']])
  return df

In [0]:
AsinNeighborsData = getCompositeScore(AsinNeighborsData)

In [17]:
AsinNeighborsData.shape

(41, 15)

In [18]:
AsinNeighborsData.sort_values(by = ['CompositeScore'], ascending=False, inplace= True)

# Print Top 5 recommendations (ASIN, and associated Title, Sales Rank, 
# TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff)
# (5) YOUR CODE HERE:  

print("Top 5 Recommendations for those who purchased ", purchasedAsin, "are.. \n")
AsinNeighborsData[['Title', 'SalesRank', 'TotalReviews', 'AvgRating', 'DegreeCentrality', 'ClusteringCoeff','CompositeScore', 'RecommendationScore']].head(5)

Top 5 Recommendations for those who purchased  0805047905 are.. 



,Title,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff,CompositeScore,RecommendationScore
067167949X,Chicka Chicka Boom Boom,5537,89,5.0,4,1.0,12.325162,1.000000
0399208534,The Very Hungry Caterpillar,2117,164,4.5,5,1.0,11.493703,0.924551
0060235152,From Head to Toe,43438,22,5.0,5,0.9,11.490446,0.924256
0399237720,Very Hungry Caterpillar Giant Board Book and P...,112356,164,4.5,5,1.0,11.169707,0.895151
0064435962,From Head to Toe,187777,22,5.0,5,0.9,11.074373,0.886500
